# AES Cipher

In [3]:
# Dependencies @base64, @hashlib, @Crypto

import base64
import hashlib

from Crypto import Random
from Crypto.Cipher import AES


class AESCipher(object):

    def __init__(self, key):
        self.bs = AES.block_size
        self.key = hashlib.sha256(key.encode()).digest()

    def encrypt(self, raw):
        raw = self._pad(raw)
        iv = Random.new().read(AES.block_size)  #Initialisation Vector
        cipher = AES.new(self.key, AES.MODE_CBC, iv)  #Cipher Block Chaining Mode
        return base64.b64encode(iv + cipher.encrypt(raw))  #binary to text encoding
        #Image will be base64 encoded, so the secret message is also encrypted to base64
        
    def decrypt(self, enc):
        enc = base64.b64decode(enc)
        iv = enc[:AES.block_size]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return self._unpad(cipher.decrypt(enc[AES.block_size:])).decode('utf-8')

    def _pad(self, s): #private
        return s + ((self.bs - len(s) % self.bs) * chr(self.bs - len(s) % self.bs)).encode()

    @staticmethod
    def _unpad(s):
        return s[:-ord(s[len(s) - 1:])]
                    


In [4]:
key ='øám`«õãtÙ¸i\x9as\x8eØ\x01'
aes = AESCipher(key)

In [5]:
aes.encrypt(b"We are a family")

b'OEOom9MLDSeFosOSMc/hXz//nVw5JdbyCzIWvOQxXHk='

In [6]:
aes.decrypt(b'gcM+zpCw12y7WDwnNneT7e6bq/VYBMuHokhph6yvQ04=')

'We are a family'

In [7]:
# keysample = ''.join(chr(random.randint(0, 0xFF)) for i in range(16))
key ='øám`«õãtÙ¸i\x9as\x8eØ\x01'
aes = AESCipher(key)

In [8]:
aes.key

b'y1\x14\xdb5\x00\x16\x89\xef\xf7\xaa\x11\xc2\x02\xd1.\xf6\x1b\x00e\xc9\x9b\xfa;N\xd6Yt\x8b1\x04\x12'

# Encryption and QR Generation / Embedding

In [35]:
# Dependencies @opencv @python_Wavelet_Transform @qrCode @numpy

import cv2
import pywt
import qrcode
import numpy as np

def aes_encrypt(key: str, data: str):
    cipher = AESCipher(key)
    enc = cipher.encrypt(data)
    return enc


def embed(cover_path: str, key: str, data: str):
    enc = aes_encrypt(key, data)

    cover = cv2.imread(cover_path)
    cover = cv2.cvtColor(cover, cv2.COLOR_BGR2GRAY)
    datacover = np.asarray(cover, dtype="float64")  #converts to numpy array
    coeff2 = pywt.dwt2(datacover, 'haar')
    print("Coefficient 2\n")
    print(coeff2)   #DWT transform Coeff
    print("\n")

    qr = qrcode.make(enc)
    
    qrc = qrcode.QRCode()
    
    qrc.add_data(enc)
    qrc.make(fit=True)
    img = qrc.make_image(fill_color="black", back_color="white").convert('RGB')
    img.save("sample.png")

    size = cover.shape #2700*2700
    qr = qr.resize((size[1], size[0]))
    qr = qr.convert("L")   #Data Correction Level(depends on modules)
    dataqr = np.asarray(qr, dtype="float64")
    coeff1 = pywt.dwt2(dataqr, 'haar')
    print("Coefficient 1\n")
    print(coeff1)    #QR Code Embedding Coefficient
    print("\n")
    
    def fuse_coeff(coeff1, coeff2):
        coeff = (coeff1 * 0.0050 + coeff2 * 0.9950)  #alpha = 0.5 beta = 99.5  
        return coeff

    stego = []
    stego.append(fuse_coeff(coeff1[0], coeff2[0]))

    c1 = fuse_coeff(coeff1[1][0], coeff2[1][0])
    c2 = fuse_coeff(coeff1[1][1], coeff2[1][1])
    c3 = fuse_coeff(coeff1[1][2], coeff2[1][2])
    stego.append((c1, c2, c3))
    print("Steganographed Image\n")
    print(stego)
    print("\n")
    cv2.imwrite("stego.png", pywt.idwt2(stego, 'haar'))



In [36]:
pywt.families(short=False)

['Haar',
 'Daubechies',
 'Symlets',
 'Coiflets',
 'Biorthogonal',
 'Reverse biorthogonal',
 'Discrete Meyer (FIR Approximation)',
 'Gaussian',
 'Mexican hat wavelet',
 'Morlet wavelet',
 'Complex Gaussian wavelets',
 'Shannon wavelets',
 'Frequency B-Spline wavelets',
 'Complex Morlet wavelets']

In [37]:
embed("image.JPG",'bcjdbcjd',b"dbxejxbjdbcjdbchdvhcd")

Coefficient 2

(array([[180., 180., 180., ..., 438., 439., 441.],
       [ 71.,  71.,  71., ..., 438., 439., 441.],
       [ 70.,  70.,  70., ..., 438., 439., 441.],
       ...,
       [ 70.,  70.,  70., ..., 116., 116., 116.],
       [ 70.,  70.,  70., ..., 116., 116., 116.],
       [ 70.,  70.,  70., ..., 116., 116., 116.]]), (array([[110., 110., 110., ...,   0.,   0.,   0.],
       [  1.,   1.,   1., ...,   0.,   0.,   0.],
       [  2.,   2.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]]), array([[ 0.,  0.,  0., ...,  0., -1., -1.],
       [ 0.,  0.,  0., ...,  0., -1., -1.],
       [ 0.,  0.,  0., ...,  0., -1., -1.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
 

# Decryption

In [10]:
import cv2
import pywt
import numpy as np



def recover(stego_path: str, cover_path: str):
    stego = cv2.imread(stego_path)
    stego = cv2.cvtColor(stego, cv2.COLOR_BGR2GRAY)
    coeff2 = pywt.dwt2(np.asarray(stego, dtype="float64"), 'haar')

    cover = cv2.imread(cover_path)
    cover = cv2.cvtColor(cover, cv2.COLOR_BGR2GRAY)
    size = stego.shape
    cover = cv2.resize(cover, (size[1], size[0]))
    coeff1 = pywt.dwt2(np.asarray(cover, dtype="float64"), 'haar')

    def fuse_coeff(coeff1, coeff2):
        cooef = (coeff2 - (coeff1 * 0.9950)) / 0.0050
        return cooef

    recovered = []
    recovered.append(fuse_coeff(coeff1[0], coeff2[0]))
    c1 = fuse_coeff(coeff1[1][0], coeff2[1][0])
    c2 = fuse_coeff(coeff1[1][1], coeff2[1][1])
    c3 = fuse_coeff(coeff1[1][2], coeff2[1][2])

    recovered.append((c1, c2, c3))
    cv2.imwrite("decoded.png", pywt.idwt2(recovered, 'haar'))


def aes_decrypt(key: str, data: str):
    cipher = AESCipher(key)
    dec = cipher.decrypt(data)
    return dec



In [9]:
Random.new().read(AES.block_size)

b'\xdeN\xafedf\xf1dX\x16r\x99\xa1\x04\x9d\xa0'

In [10]:
a =b'\xdeN\xafedf\xf1dX\x16r\x99\xa1\x04\x9d\xa0'

In [11]:
len(a)

16